In [1]:
import torch
torch.cuda.set_device(1)
print('cuda:', torch.cuda.current_device())
device = "cuda" if torch.cuda.is_available() else "cpu"

cuda: 1


# Data

## Dataset
- Original dataset is [Voxceleb1](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/).
- The [license](https://creativecommons.org/licenses/by/4.0/) and [complete version](https://www.robots.ox.ac.uk/~vgg/data/voxceleb/files/license.txt) of Voxceleb1.
- We randomly select 600 speakers from Voxceleb1.
- Then preprocess the raw waveforms into mel-spectrograms.

- Args:
  - data_dir: The path to the data directory.
  - metadata_path: The path to the metadata.
  - segment_len: The length of audio segment for training. 
- The architecture of data directory \\
  - data directory \\
  |---- metadata.json \\
  |---- testdata.json \\
  |---- mapping.json \\
  |---- uttr-{random string}.pt \\

- The information in metadata
  - "n_mels": The dimention of mel-spectrogram.
  - "speakers": A dictionary. 
    - Key: speaker ids.
    - value: "feature_path" and "mel_len"


For efficiency, we segment the mel-spectrograms into segments in the traing step.

In [2]:
segment_len = 128
d_model = 160
conform_dropout = 0.1
sa_dim = 8
model_name = 'HW04-conform-sa8_dmodel160'

In [3]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class myDataset(Dataset):
    def __init__(self, data_dir, segment_len=128):
        self.data_dir = data_dir
        self.segment_len = segment_len

        # Load the mapping from speaker neme to their corresponding id.
        mapping_path = Path(data_dir) / "mapping.json"
        mapping = json.load(mapping_path.open())
        self.speaker2id = mapping["speaker2id"]

        # Load metadata of training data.
        metadata_path = Path(data_dir) / "metadata.json"
        metadata = json.load(open(metadata_path))["speakers"]

        # Get the total number of speaker.
        self.speaker_num = len(metadata.keys())
        self.data = []
        for speaker in metadata.keys():
            for utterances in metadata[speaker]:
                self.data.append(
                    [utterances["feature_path"], self.speaker2id[speaker]])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        feat_path, speaker = self.data[index]
        # Load preprocessed mel-spectrogram.
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        # Segmemt mel-spectrogram into "segment_len" frames.
        if self.segment_len == -1:
            mel = torch.FloatTensor(mel)
        elif len(mel) > self.segment_len:
            # Randomly get the starting point of the segment.
            start = random.randint(0, len(mel) - self.segment_len)
            # Get a segment with "segment_len" frames.
            mel = torch.FloatTensor(mel[start:start + self.segment_len])
        else:
            mel = torch.FloatTensor(mel)
            
        # Turn the speaker id into long for computing loss later.
        speaker = torch.FloatTensor([speaker]).long()
        return mel, speaker

    def get_speaker_number(self):
        return self.speaker_num

## Dataloader
- Split dataset into training dataset(90%) and validation dataset(10%).
- Create dataloader to iterate the data.


In [4]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
    # Process features within a batch.
    """Collate a batch of data."""
    mel, speaker = zip(*batch)
    # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
    mel = pad_sequence(
        mel, batch_first=True,
        padding_value=-20)  # pad log 10^(-20) which is very small value.
    # mel: (batch size, length, 40)
    return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    """Generate dataloader"""
    dataset = myDataset(data_dir, segment_len)
    speaker_num = dataset.get_speaker_number()
    # Split dataset into training dataset and validation dataset
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        num_workers=n_workers,
        drop_last=True,
        pin_memory=True,
        collate_fn=collate_batch,
    )

    return train_loader, valid_loader, speaker_num

# Model
- TransformerEncoderLayer:
  - Base transformer encoder layer in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
  - Parameters:
    - d_model: the number of expected features of the input (required).

    - nhead: the number of heads of the multiheadattention models (required).

    - dim_feedforward: the dimension of the feedforward network model (default=2048).

    - dropout: the dropout value (default=0.1).

    - activation: the activation function of intermediate layer, relu or gelu (default=relu).

- TransformerEncoder:
  - TransformerEncoder is a stack of N transformer encoder layers
  - Parameters:
    - encoder_layer: an instance of the TransformerEncoderLayer() class (required).

    - num_layers: the number of sub-encoder-layers in the encoder (required).

    - norm: the layer normalization component (optional).

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
#         print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
        out = out.view(32, -1)      
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
    


# Learning rate schedule
- For transformer architecture, the design of learning rate schedule is different from that of CNN.
- Previous works show that the warmup of learning rate is useful for training models with transformer architectures.
- The warmup schedule
  - Set learning rate to 0 in the beginning.
  - The learning rate increases linearly from 0 to initial learning rate during warmup period.

In [6]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
):
    """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """
    def lr_lambda(current_step):
        # Warmup
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        # decadence
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps))
        return max(
            0.0, 0.5 *
            (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))

    return LambdaLR(optimizer, lr_lambda, last_epoch)

# Model Function
- Model forward function.

In [7]:
import torch


def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)

    outs = model(mels)

    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

# Validate
- Calculate accuracy of the validation set.

In [8]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device):
    """Validate on validation set."""

    model.eval()
    running_loss = 0.0
    running_accuracy = 0.0
    pbar = tqdm(total=len(dataloader.dataset),
                ncols=0,
                desc="Valid",
                unit=" uttr")

    for i, batch in enumerate(dataloader):
        with torch.no_grad():
            loss, accuracy = model_fn(batch, model, criterion, device)
            running_loss += loss.item()
            running_accuracy += accuracy.item()

        pbar.update(dataloader.batch_size)
        pbar.set_postfix(
            loss=f"{running_loss / (i+1):.2f}",
            accuracy=f"{running_accuracy / (i+1):.2f}",
        )

    pbar.close()
    model.train()

    return running_accuracy / len(dataloader)

# Main function

In [9]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
    """arguments"""
    config = {
        "data_dir": "./Dataset",
        "save_path": model_name,
        "batch_size": 32,
        "n_workers": 2,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 10000,
        "total_steps": 120000,
    }

    return config


def main(
    data_dir,
    save_path,
    batch_size,
    n_workers,
    valid_steps,
    warmup_steps,
    total_steps,
    save_steps,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    train_loader, valid_loader, speaker_num = get_dataloader(
        data_dir, batch_size, n_workers)
    train_iterator = iter(train_loader)
    print(f"[Info]: Finish loading data!", flush=True)

    model = Classifier(n_spks=speaker_num).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps,
                                                total_steps)
    print(f"[Info]: Finish creating model!", flush=True)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    for step in range(total_steps):
        # Get data
        try:
            batch = next(train_iterator)
        except StopIteration:
            train_iterator = iter(train_loader)
            batch = next(train_iterator)

        loss, accuracy = model_fn(batch, model, criterion, device)
        batch_loss = loss.item()
        batch_accuracy = accuracy.item()

        # Updata model
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        # Log
        pbar.update()
        pbar.set_postfix(
            loss=f"{batch_loss:.2f}",
            accuracy=f"{batch_accuracy:.2f}",
            step=step + 1,
        )

        # Do validation
        if (step + 1) % valid_steps == 0:
            pbar.close()

            valid_accuracy = valid(valid_loader, model, criterion, device)

            # keep the best model
            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()

            pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        # Save the best model so far.
        if (step + 1) % save_steps == 0 and best_state_dict is not None:
            torch.save(best_state_dict, save_path)
            pbar.write(
                f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})"
            )

    pbar.close()


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


Train: 100% 2000/2000 [01:13<00:00, 27.05 step/s, accuracy=0.34, loss=3.04, step=2000]
Valid: 100% 6944/6944 [00:02<00:00, 2653.21 uttr/s, accuracy=0.46, loss=2.53]
Train: 100% 2000/2000 [01:24<00:00, 23.61 step/s, accuracy=0.72, loss=1.27, step=4000]
Valid: 100% 6944/6944 [00:02<00:00, 2645.53 uttr/s, accuracy=0.58, loss=1.88]
Train: 100% 2000/2000 [01:25<00:00, 23.47 step/s, accuracy=0.78, loss=0.93, step=6000]
Valid: 100% 6944/6944 [00:02<00:00, 2726.42 uttr/s, accuracy=0.65, loss=1.52]
Train: 100% 2000/2000 [01:24<00:00, 23.73 step/s, accuracy=0.66, loss=1.24, step=8000]
Valid: 100% 6944/6944 [00:02<00:00, 2690.01 uttr/s, accuracy=0.67, loss=1.42]
Train: 100% 2000/2000 [01:25<00:00, 23.42 step/s, accuracy=0.81, loss=0.99, step=1e+4]
Valid: 100% 6944/6944 [00:02<00:00, 2661.68 uttr/s, accuracy=0.72, loss=1.20]
Train:   0% 5/2000 [00:00<01:36, 20.60 step/s, accuracy=0.81, loss=0.88, step=1e+4]

Step 10000, best model saved. (accuracy=0.7185)


Train: 100% 2000/2000 [01:26<00:00, 23.16 step/s, accuracy=0.78, loss=0.90, step=12000]
Valid: 100% 6944/6944 [00:02<00:00, 2676.26 uttr/s, accuracy=0.74, loss=1.09]
Train: 100% 2000/2000 [01:24<00:00, 23.68 step/s, accuracy=0.75, loss=0.67, step=14000]
Valid: 100% 6944/6944 [00:02<00:00, 2715.63 uttr/s, accuracy=0.74, loss=1.10]
Train: 100% 2000/2000 [01:29<00:00, 22.42 step/s, accuracy=0.88, loss=0.62, step=16000]
Valid: 100% 6944/6944 [00:02<00:00, 2596.16 uttr/s, accuracy=0.77, loss=0.99]
Train: 100% 2000/2000 [01:28<00:00, 22.57 step/s, accuracy=0.97, loss=0.18, step=18000]
Valid: 100% 6944/6944 [00:02<00:00, 2736.94 uttr/s, accuracy=0.78, loss=0.96]
Train: 100% 2000/2000 [01:27<00:00, 22.91 step/s, accuracy=0.94, loss=0.26, step=2e+4] 
Valid: 100% 6944/6944 [00:02<00:00, 2772.63 uttr/s, accuracy=0.78, loss=0.92]
Train:   0% 5/2000 [00:00<01:23, 23.96 step/s, accuracy=0.78, loss=0.63, step=2e+4]

Step 20000, best model saved. (accuracy=0.7820)


...

Step 100000, best model saved. (accuracy=0.9057)


Train: 100% 2000/2000 [01:23<00:00, 23.92 step/s, accuracy=1.00, loss=0.03, step=102000]
Valid: 100% 6944/6944 [00:02<00:00, 2669.02 uttr/s, accuracy=0.90, loss=0.45]
Train: 100% 2000/2000 [01:23<00:00, 23.82 step/s, accuracy=1.00, loss=0.01, step=104000]
Valid: 100% 6944/6944 [00:02<00:00, 2694.80 uttr/s, accuracy=0.91, loss=0.44]
Train: 100% 2000/2000 [01:25<00:00, 23.49 step/s, accuracy=1.00, loss=0.01, step=106000]
Valid: 100% 6944/6944 [00:02<00:00, 2812.54 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [01:27<00:00, 22.92 step/s, accuracy=1.00, loss=0.01, step=108000]
Valid: 100% 6944/6944 [00:02<00:00, 2783.45 uttr/s, accuracy=0.91, loss=0.45]
Train: 100% 2000/2000 [01:26<00:00, 23.03 step/s, accuracy=1.00, loss=0.01, step=110000]
Valid: 100% 6944/6944 [00:02<00:00, 2669.75 uttr/s, accuracy=0.91, loss=0.44]
Train:   0% 4/2000 [00:00<02:15, 14.68 step/s, accuracy=1.00, loss=0.02, step=110004]

Step 110000, best model saved. (accuracy=0.9086)


Train: 100% 2000/2000 [01:27<00:00, 22.96 step/s, accuracy=1.00, loss=0.01, step=112000]
Valid: 100% 6944/6944 [00:02<00:00, 2771.35 uttr/s, accuracy=0.91, loss=0.44]
Train: 100% 2000/2000 [01:26<00:00, 23.03 step/s, accuracy=1.00, loss=0.01, step=114000]
Valid: 100% 6944/6944 [00:02<00:00, 2745.95 uttr/s, accuracy=0.90, loss=0.45]
Train: 100% 2000/2000 [01:25<00:00, 23.29 step/s, accuracy=1.00, loss=0.00, step=116000]
Valid: 100% 6944/6944 [00:02<00:00, 2752.70 uttr/s, accuracy=0.91, loss=0.43]
Train: 100% 2000/2000 [01:24<00:00, 23.58 step/s, accuracy=1.00, loss=0.01, step=118000]
Valid: 100% 6944/6944 [00:02<00:00, 2578.52 uttr/s, accuracy=0.90, loss=0.46]
Train: 100% 2000/2000 [01:09<00:00, 28.90 step/s, accuracy=1.00, loss=0.04, step=120000]
Valid: 100% 6944/6944 [00:01<00:00, 3649.49 uttr/s, accuracy=0.91, loss=0.45]
Train:   0% 0/2000 [00:00<?, ? step/s]


Step 120000, best model saved. (accuracy=0.9086)


# Inference

## Dataset of inference

In [10]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
    def __init__(self, data_dir):
        testdata_path = Path(data_dir) / "testdata.json"
        metadata = json.load(testdata_path.open())
        self.data_dir = data_dir
        self.data = metadata["utterances"]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        utterance = self.data[index]
        feat_path = utterance["feature_path"]
        mel = torch.load(os.path.join(self.data_dir, feat_path))

        return feat_path, mel


def inference_collate_batch(batch):
    """Collate a batch of data."""
    feat_paths, mels = zip(*batch)

    return feat_paths, torch.stack(mels)

## Main funcrion of Inference

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
        print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
#         out = out.view(-1, d_model)  
        out = out.view(32, -1)  
        print(out.shape)
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
a = torch.randn(32,150,40)
model = Classifier()
# print(model)
model(a).shape

A torch.Size([32, 8, 150])
torch.Size([32, 1280])


torch.Size([32, 600])

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from conformer import ConformerBlock

class Classifier(nn.Module):
    def __init__(self, d_model=d_model, n_spks=600, dropout=0.1, sa_dim=sa_dim):
        super().__init__()
        # Project the dimension of features from that of input into d_model.
        self.prenet = nn.Linear(40, d_model)
        # TODO:
        #   Change Transformer to Conformer.
        #   https://arxiv.org/abs/2005.08100
#         self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model,
#                                                         dim_feedforward=256,
#                                                         nhead=2)
        self.encoder_layer = ConformerBlock(
                        dim = d_model,
                        dim_head = 64,
                        heads = 8,
                        ff_mult = 4,
                        conv_expansion_factor = 2,
                        conv_kernel_size = 31,
                        attn_dropout = conform_dropout,
                        ff_dropout = conform_dropout,
                        conv_dropout = conform_dropout
                    )
        # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)
        self.sa_layer1 = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(),
        )   

        self.sa_layer2 = nn.Sequential(
            nn.Linear(d_model, sa_dim),
            nn.Softmax(dim=1),
        )  
        # Project the the dimension of features from d_model into speaker nums.
        self.pred_layer = nn.Sequential(
#             nn.Linear(d_model, d_model),
#             nn.ReLU(),
            nn.Linear(d_model*sa_dim, n_spks),
        )

    def forward(self, mels):
        """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
        # out: (batch size, length, d_model)
        out = self.prenet(mels)
        # out: (length, batch size, d_model)
        out = out.permute(1, 0, 2)
        # The encoder layer expect features in the shape of (length, batch size, d_model).
        out = self.encoder_layer(out)
        # out: (batch size, length, d_model)
        out = out.transpose(0, 1)
        
        a1 = self.sa_layer1(out)
        A = self.sa_layer2(a1)        
        A = A.permute(0, 2, 1)
        print('A',A.shape)
#         print('out',out.shape)
        
        out = torch.matmul(A, out)
#         out = out.view(-1, d_model)  
        out = out.view(1, -1)  
        print(out.shape)
#         out = out.mean(dim=1)        
        # out: (batch, n_spks)
        out = self.pred_layer(out)
        return out
a = torch.randn(1,150,40)
model = Classifier()
# print(model)
model(a).shape

A torch.Size([1, 8, 150])
torch.Size([1, 1280])


torch.Size([1, 600])

In [13]:
import json
import csv
from pathlib import Path
from tqdm.notebook import tqdm

import torch
from torch.utils.data import DataLoader


def parse_args():
    """arguments"""
    config = {
        "data_dir": "./Dataset",
        "model_path": model_name,
        "output_path": "./output_{}.csv".format(model_name),
    }

    return config


def main(
    data_dir,
    model_path,
    output_path,
):
    """Main function."""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"[Info]: Use {device} now!")

    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())

    dataset = InferenceDataset(data_dir)
    dataloader = DataLoader(
        dataset,
        batch_size=1,
        shuffle=False,
        drop_last=False,
        num_workers=8,
        collate_fn=inference_collate_batch,
    )
    print(f"[Info]: Finish loading data!", flush=True)

    speaker_num = len(mapping["id2speaker"])
    model = Classifier(n_spks=speaker_num).to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    print(f"[Info]: Finish creating model!", flush=True)

    results = [["Id", "Category"]]
    for feat_paths, mels in tqdm(dataloader):
        with torch.no_grad():
            mels = mels.to(device)
            print(mels.shape)
            outs = model(mels)
            print(outs.shape)
            preds = outs.argmax(1).cpu().numpy()
            for feat_path, pred in zip(feat_paths, preds):
                results.append([feat_path, mapping["id2speaker"][str(pred)]])

    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(results)


if __name__ == "__main__":
    main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!


  0%|          | 0/6000 [00:00<?, ?it/s]

torch.Size([1, 2112, 40])
A torch.Size([1, 8, 2112])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 586, 40])
A torch.Size([1, 8, 586])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 465, 40])
A torch.Size([1, 8, 465])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 404, 40])
A torch.Size([1, 8, 404])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 402, 40])
A torch.Size([1, 8, 402])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 428, 40])
A torch.Size([1, 8, 428])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 899, 40])
A torch.Size([1, 8, 899])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 1677, 40])
A torch.Size([1, 8, 1677])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 335, 40])
A torch.Size([1, 8, 335])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 528, 40])
A torch.Size([1, 8, 528])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 533, 40])
A torch.Size([1, 8, 533])
torch

torch.Size([1, 2048, 40])
A torch.Size([1, 8, 2048])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 729, 40])
A torch.Size([1, 8, 729])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 1133, 40])
A torch.Size([1, 8, 1133])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 424, 40])
A torch.Size([1, 8, 424])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 361, 40])
A torch.Size([1, 8, 361])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 616, 40])
A torch.Size([1, 8, 616])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 594, 40])
A torch.Size([1, 8, 594])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 890, 40])
A torch.Size([1, 8, 890])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 400, 40])
A torch.Size([1, 8, 400])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 404, 40])
A torch.Size([1, 8, 404])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 556, 40])
A torch.Size([1, 8, 556])
torch

torch.Size([1, 1289, 40])
A torch.Size([1, 8, 1289])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 505, 40])
A torch.Size([1, 8, 505])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 488, 40])
A torch.Size([1, 8, 488])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 716, 40])
A torch.Size([1, 8, 716])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 395, 40])
A torch.Size([1, 8, 395])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 430, 40])
A torch.Size([1, 8, 430])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 667, 40])
A torch.Size([1, 8, 667])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 344, 40])
A torch.Size([1, 8, 344])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 445, 40])
A torch.Size([1, 8, 445])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 476, 40])
A torch.Size([1, 8, 476])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 622, 40])
A torch.Size([1, 8, 622])
torch.S

...

A torch.Size([1, 8, 722])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 372, 40])
A torch.Size([1, 8, 372])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 373, 40])
A torch.Size([1, 8, 373])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 452, 40])
A torch.Size([1, 8, 452])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 1209, 40])
A torch.Size([1, 8, 1209])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 381, 40])
A torch.Size([1, 8, 381])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 393, 40])
A torch.Size([1, 8, 393])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 500, 40])
A torch.Size([1, 8, 500])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 876, 40])
A torch.Size([1, 8, 876])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 597, 40])
A torch.Size([1, 8, 597])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 985, 40])
A torch.Size([1, 8, 985])
torch.Size([1, 1280])
torch.Size

A torch.Size([1, 8, 562])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 485, 40])
A torch.Size([1, 8, 485])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 298, 40])
A torch.Size([1, 8, 298])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 479, 40])
A torch.Size([1, 8, 479])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 460, 40])
A torch.Size([1, 8, 460])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 2516, 40])
A torch.Size([1, 8, 2516])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 641, 40])
A torch.Size([1, 8, 641])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 456, 40])
A torch.Size([1, 8, 456])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 1151, 40])
A torch.Size([1, 8, 1151])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 420, 40])
A torch.Size([1, 8, 420])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 776, 40])
A torch.Size([1, 8, 776])
torch.Size([1, 1280])
torch.Si

A torch.Size([1, 8, 388])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 458, 40])
A torch.Size([1, 8, 458])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 406, 40])
A torch.Size([1, 8, 406])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 953, 40])
A torch.Size([1, 8, 953])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 632, 40])
A torch.Size([1, 8, 632])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 764, 40])
A torch.Size([1, 8, 764])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 560, 40])
A torch.Size([1, 8, 560])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 254, 40])
A torch.Size([1, 8, 254])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 860, 40])
A torch.Size([1, 8, 860])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 497, 40])
A torch.Size([1, 8, 497])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 352, 40])
A torch.Size([1, 8, 352])
torch.Size([1, 1280])
torch.Size([

torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 512, 40])
A torch.Size([1, 8, 512])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 2733, 40])
A torch.Size([1, 8, 2733])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 390, 40])
A torch.Size([1, 8, 390])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 1640, 40])
A torch.Size([1, 8, 1640])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 1664, 40])
A torch.Size([1, 8, 1664])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 690, 40])
A torch.Size([1, 8, 690])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 430, 40])
A torch.Size([1, 8, 430])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 426, 40])
A torch.Size([1, 8, 426])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 892, 40])
A torch.Size([1, 8, 892])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 397, 40])
A torch.Size([1, 8, 397])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size(

torch.Size([1, 600])
torch.Size([1, 396, 40])
A torch.Size([1, 8, 396])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 444, 40])
A torch.Size([1, 8, 444])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 727, 40])
A torch.Size([1, 8, 727])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 776, 40])
A torch.Size([1, 8, 776])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 575, 40])
A torch.Size([1, 8, 575])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 608, 40])
A torch.Size([1, 8, 608])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 630, 40])
A torch.Size([1, 8, 630])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 876, 40])
A torch.Size([1, 8, 876])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 583, 40])
A torch.Size([1, 8, 583])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 1930, 40])
A torch.Size([1, 8, 1930])
torch.Size([1, 1280])
torch.Size([1, 600])
torch.Size([1, 604, 40])
A torch.Size